In [3]:
from azureml.core import Workspace, Dataset, Datastore
import os
from azureml.core.authentication import InteractiveLoginAuthentication
from azureml.core import Run

import azureml.core
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core import ComputeTarget

from azureml.pipeline.steps import PythonScriptStep
from azureml.core import Experiment, Environment
from azureml.core.runconfig import RunConfiguration
from azureml.pipeline.core import Pipeline



In [3]:
pip install azureml-pipeline

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.7/313.7 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.6/69.6 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 73.0 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.6/137.6 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 122.5 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 249.6/249.6 kB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 kB 3.5 MB/s eta 0:00:00
  Attempting uninstall: azureml-core
    Found existing installation: azureml-core 1.56.0
    Uninstalling azureml-core-1.56.0:
      Successfully uninstalled azureml-core-1.56.0
Note: you may need to restart the kernel to use updated packages.


In [17]:
ws = Workspace.from_config("Users/mypersonall3099/Online_retail/config.json") 


In [75]:
compute_name = "OnlineRetail"
#vm_size = "Standard_E4ds_v4"
vm_size = "Standard_E8ds_v4"
compute_target = ws.compute_targets[compute_name]

In [76]:
from azureml.core import Environment

env = Environment.from_conda_specification(name="myenv", file_path="environment.yaml")
env.register(workspace=ws)


{
    "assetId": "azureml://locations/eastus2/workspaces/7ce76f32-f5fe-4dd3-b900-96fb92b58ebb/environments/myenv/versions/1",
    "databricks": {
        "eggLibraries": [],
        "jarLibraries": [],
        "mavenLibraries": [],
        "pypiLibraries": [],
        "rcranLibraries": []
    },
    "docker": {
        "arguments": [],
        "baseDockerfile": null,
        "baseImage": "mcr.microsoft.com/azureml/openmpi4.1.0-ubuntu20.04:20240709.v1",
        "baseImageRegistry": {
            "address": null,
            "password": null,
            "registryIdentity": null,
            "username": null
        },
        "buildContext": null,
        "enabled": false,
        "platform": {
            "architecture": "amd64",
            "os": "Linux"
        },
        "sharedVolumes": true,
        "shmSize": null
    },
    "environmentVariables": {
        "EXAMPLE_ENV_VAR": "EXAMPLE_VALUE"
    },
    "inferencingStackVersion": null,
    "name": "myenv",
    "python": {
       

In [77]:
from azureml.core import Environment

env = Environment.get(workspace=ws, name="myenv")
print(env.python.conda_dependencies.serialize_to_string())


name: myenv
channels:
- defaults
dependencies:
- python=3.7
- pandas
- scikit-learn
- statsmodels
- pip
- pip:
  - pmdarima
  - azureml-sdk
  - azureml-dataset-runtime[fuse, pandas]
  - seaborn
  - pandas



In [78]:
#Declaring environment
aml_config = RunConfiguration()
aml_config.target = compute_target 

env = Environment.from_conda_specification(name="myenv", file_path="environment.yaml")

In [79]:
#Pipeline
read_data = 'data_wrangling.py'
prep = 'preprocessing.py'
model = 'modelling.py'

#Script initialization
py_script_run_read = PythonScriptStep(
                script_name = read_data,
                compute_target = compute_target,
                arguments=['--input-data','online_retail_II.csv'],
                runconfig = aml_config,
                allow_reuse=False)

py_script_run_prep = PythonScriptStep(
                script_name = prep,
                compute_target=compute_target,
                arguments=['--prep','wranggled.csv'],
                runconfig = aml_config,
                allow_reuse=False)

py_script_run_model = PythonScriptStep(
                script_name = model,
                compute_target=compute_target,
                arguments=['--train','preprocessed.csv'],
                runconfig = aml_config,
                allow_reuse=False)

pipeline_steps = [py_script_run_read, py_script_run_prep, py_script_run_model]
pipeline_1 = Pipeline(workspace=ws, steps = pipeline_steps)

In [81]:
# Experiment

pipeline_run = Experiment(ws, "First_run_7").submit(pipeline_1)
pipeline_run.wait_for_completion(show_output=True)

Submitted PipelineRun c6cc82fe-2d0a-425b-b534-bbb885e45576
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/c6cc82fe-2d0a-425b-b534-bbb885e45576?wsid=/subscriptions/d8c4fe8f-f720-48ac-86e8-880576d9cd7a/resourcegroups/OnlineRetail/workspaces/OnlineRetail&tid=f56f1f69-458e-427b-bada-4cba658f7917
PipelineRunId: c6cc82fe-2d0a-425b-b534-bbb885e45576
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/c6cc82fe-2d0a-425b-b534-bbb885e45576?wsid=/subscriptions/d8c4fe8f-f720-48ac-86e8-880576d9cd7a/resourcegroups/OnlineRetail/workspaces/OnlineRetail&tid=f56f1f69-458e-427b-bada-4cba658f7917
PipelineRun Status: NotStarted
PipelineRun Status: Running


StepRunId: b8b03587-b6ca-4dd3-a0d4-f48947d08d8e
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/b8b03587-b6ca-4dd3-a0d4-f48947d08d8e?wsid=/subscriptions/d8c4fe8f-f720-48ac-86e8-880576d9cd7a/resourcegroups/OnlineRetail/workspaces/OnlineRetail&tid=f56f1f69-458e-427b-bada-4cba658f7917
StepRun( preprocessing.p

ActivityFailedException: ActivityFailedException:
	Message: Activity Failed:
{
    "error": {
        "code": "UserError",
        "message": "Execution failed. User process '/azureml-envs/azureml_f3f7e6c5fb83d94df23933000bf02da3/bin/python' exited with status code 1. Please check log file 'user_logs/std_log.txt' for error details. Error: Traceback (most recent call last):\n  File \"preprocessing.py\", line 2, in <module>\n    import pandas as pd\nModuleNotFoundError: No module named 'pandas'\n\n",
        "messageParameters": {},
        "details": []
    },
    "time": "0001-01-01T00:00:00.000Z",
    "componentName": "CommonRuntime"
}
	InnerException None
	ErrorResponse 
{
    "error": {
        "message": "Activity Failed:\n{\n    \"error\": {\n        \"code\": \"UserError\",\n        \"message\": \"Execution failed. User process '/azureml-envs/azureml_f3f7e6c5fb83d94df23933000bf02da3/bin/python' exited with status code 1. Please check log file 'user_logs/std_log.txt' for error det

In [ ]:
# Experiment 2

pipeline_run = Experiment(ws, "Second_run").submit(pipeline_1)
pipeline_run.wait_for_completion(show_output=True)